# Import important packages

In [ ]:
#Install Kaggle library
pip install kaggle

# - Log in to kaggle account
# - From the my account tab, download the API Token. Place it in "C:\Users\Hi\.kaggle" folder

In [1]:
import pandas as pd, numpy as np, scipy as sp, sklearn,requests,kaggle, matplotlib.pyplot as plt, seaborn as sb,plotly as ply, statsmodels as stat, tabulate,re,nltk

# Download data using Kaggle web API

In [4]:
#Import datafiles from kaggle API
!kaggle competitions download -c home-credit-default-risk -f application_train.csv
!kaggle competitions download -c home-credit-default-risk -f bureau.csv
!kaggle competitions download -c home-credit-default-risk -f bureau_balance.csv
!kaggle competitions download -c home-credit-default-risk -f credit_card_balance.csv
!kaggle competitions download -c home-credit-default-risk -f installments_payments.csv
!kaggle competitions download -c home-credit-default-risk -f POS_CASH_balance.csv
!kaggle competitions download -c home-credit-default-risk -f previous_application.csv


  0%|          | 0.00/36.1M [00:00<?, ?B/s]
  3%|2         | 1.00M/36.1M [00:00<00:06, 5.72MB/s]
 11%|#1        | 4.00M/36.1M [00:00<00:04, 7.34MB/s]
 19%|#9        | 7.00M/36.1M [00:00<00:03, 9.26MB/s]
 25%|##4       | 9.00M/36.1M [00:00<00:02, 11.0MB/s]
 30%|###       | 11.0M/36.1M [00:00<00:02, 12.3MB/s]
 36%|###6      | 13.0M/36.1M [00:00<00:01, 12.8MB/s]
 42%|####1     | 15.0M/36.1M [00:00<00:01, 14.5MB/s]
 50%|####9     | 18.0M/36.1M [00:01<00:01, 16.5MB/s]
 58%|#####8    | 21.0M/36.1M [00:01<00:00, 17.9MB/s]
 66%|######6   | 24.0M/36.1M [00:01<00:00, 19.1MB/s]
 72%|#######2  | 26.0M/36.1M [00:01<00:00, 18.1MB/s]
 80%|########  | 29.0M/36.1M [00:01<00:00, 19.7MB/s]
 89%|########8 | 32.0M/36.1M [00:01<00:00, 21.0MB/s]
 97%|#########6| 35.0M/36.1M [00:01<00:00, 21.9MB/s]
100%|##########| 36.1M/36.1M [00:01<00:00, 19.7MB/s]


The datasets are unzipped and stored at the local folder

# Data exploration

In [5]:
# Set working directory
import os
os.chdir("C:\\Users\\Hi\\Documents\\UC BANA Master\\1. Coursework\\7. Spring - 8090 Python\\Final Project\\Home Credit Data")

## Application_train

This dataset contains static data on all applications. One row represents one loan in our data sample

In [7]:
apptrain = pd.read_csv('application_train.csv')
print(apptrain.shape)
print(apptrain['SK_ID_CURR'].nunique())

(307511, 122)
307511


### Column data types

In [8]:
# Check column datatypes
apptrain.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                  int64
FLAG_WORK_PHONE                 int64
FLAG_CONT_MOBILE                int64
FLAG_PHONE  

### response variable distribution

In [10]:
# Check response variable distribution
apptrain["TARGET"].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

### Numeric variable summary stats

In [11]:
apptrain.drop(['SK_ID_CURR','TARGET'],axis=1).describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,3522.886321,1509.450419,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,-7479.500000,-4299.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,-4504.000000,-3254.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,-2010.000000,-1720.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000
